# Imports

In [1]:
import os
import logging
import importlib
importlib.reload(logging)
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s |: %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import mne
from mne.io import concatenate_raws
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import preprocessing
import numpy as np

import pickle
import json


from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from typing import Iterable, Union
import tensorflow as tf
import keras

# Functions

In [6]:
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000)) 

def create_model(n_classes=2, n_channs=19, samples = 306, dropout_rate=0.5):
    model = Sequential()
    model.add(Conv2D(40, (1, 13), input_shape=(n_channs, samples, 1), kernel_constraint = max_norm(2., axis=(0,1,2))))
    model.add(Conv2D(40, (n_channs, 1), use_bias=False, kernel_constraint = max_norm(2., axis=(0,1,2))))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
    model.add(Activation(square))
    model.add(AveragePooling2D(pool_size=(1, 35), strides=(1, 7)))
    model.add(Activation(log))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(n_classes, kernel_constraint = max_norm(0.5), activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

def train_model(path='./processed_data/', model_path = './models/', data_type='standardization_one', cuda=True, epochs=15, batch_size=16, multiplier=1, data_type_out=None):
    path = f'{path}{data_type}/'
    
    X_test = np.load(path + f'X_test.npy')
    y_test = np.load(path + f'y_test.npy')
    X_train = np.load(path + f'X_train.npy')
    y_train = np.load(path + f'y_train.npy')
    X_valid = np.load(path + f'X_valid.npy')
    y_valid = np.load(path + f'y_valid.npy')
    
    X_test = (X_test * multiplier).astype(np.float32)
    X_train = (X_train * multiplier).astype(np.float32)
    X_valid = (X_valid * multiplier).astype(np.float32)
    y_test = y_test.astype(np.int64)
    y_train = y_train.astype(np.int64)
    y_valid = y_valid.astype(np.int64)
    
    X = np.concatenate((X_train,X_test))
    y = np.concatenate((y_train,y_test))
    
    y = tf.keras.utils.to_categorical(y, num_classes=2, dtype='float32')
    y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=2, dtype='float32')
    
    X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))
    X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1))
    
    model = create_model()
    history = model.fit(X, y, batch_size = batch_size, epochs = epochs, validation_data=(X_valid, y_valid))
    
    if data_type_out:
        data_type = data_type_out
    model_path = f'{model_path}{data_type}'
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    model.save(model_path)
    history_dict = history.history
    json.dump(history_dict, open(f'{model_path}/history', 'w'))

## Classification clean

In [9]:
train_model(data_type='clean', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
903/903 [==============================] - 24s 26ms/step - loss: 0.8034 - accuracy: 0.5231 - val_loss: 0.7052 - val_accuracy: 0.5487
Epoch 2/50
903/903 [==============================] - 25s 27ms/step - loss: 0.7476 - accuracy: 0.5530 - val_loss: 0.6859 - val_accuracy: 0.5611
Epoch 3/50
903/903 [==============================] - 24s 26ms/step - loss: 0.7460 - accuracy: 0.5663 - val_loss: 0.6957 - val_accuracy: 0.5683
Epoch 4/50
903/903 [==============================] - 24s 26ms/step - loss: 0.7267 - accuracy: 0.5835 - val_loss: 0.7897 - val_accuracy: 0.5429
Epoch 5/50
903/903 [==============================] - 23s 26ms/step - loss: 0.7452 - accuracy: 0.5684 - val_loss: 0.7397 - val_accuracy: 0.5523
Epoch 6/50
903/903 [==============================] - 29s 32ms/step - loss: 0.7424 - accuracy: 0.5661 - val_loss: 0.6935 - val_accuracy: 0.5818
Epoch 7/50
903/903 [==============================] - 26s 29ms/step - loss: 0.7241 - accuracy: 0.5770 - val_loss: 0.7407 - val_accuracy:

## Classification xDawn

In [ ]:
train_model(data_type='xDawn', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
903/903 [==============================] - 29s 32ms/step - loss: 0.8166 - accuracy: 0.5206 - val_loss: 0.7130 - val_accuracy: 0.5473
Epoch 2/50
903/903 [==============================] - 26s 29ms/step - loss: 0.7677 - accuracy: 0.5417 - val_loss: 0.8384 - val_accuracy: 0.5101
Epoch 3/50
903/903 [==============================] - 27s 30ms/step - loss: 0.7565 - accuracy: 0.5478 - val_loss: 0.6983 - val_accuracy: 0.5702
Epoch 4/50
903/903 [==============================] - 29s 32ms/step - loss: 0.7605 - accuracy: 0.5384 - val_loss: 0.7155 - val_accuracy: 0.5526
Epoch 5/50
903/903 [==============================] - 29s 32ms/step - loss: 0.7598 - accuracy: 0.5465 - val_loss: 0.7451 - val_accuracy: 0.5550
Epoch 6/50
903/903 [==============================] - 28s 30ms/step - loss: 0.7539 - accuracy: 0.5597 - val_loss: 0.6991 - val_accuracy: 0.5699
Epoch 7/50
903/903 [==============================] - 26s 29ms/step - loss: 0.7526 - accuracy: 0.5538 - val_loss: 0.7024 - val_accuracy:

## Classification standardization common

In [ ]:
train_model(data_type='standardization_common', epochs=50, batch_size=16, multiplier=10)

## Classification clean participants

In [15]:
train_model(data_type='clean_participants', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
937/937 [==============================] - 30s 31ms/step - loss: 0.7931 - accuracy: 0.5223 - val_loss: 0.6908 - val_accuracy: 0.5631
Epoch 2/50
937/937 [==============================] - 28s 30ms/step - loss: 0.7576 - accuracy: 0.5357 - val_loss: 0.6899 - val_accuracy: 0.5595
Epoch 3/50
937/937 [==============================] - 27s 29ms/step - loss: 0.7477 - accuracy: 0.5501 - val_loss: 0.6836 - val_accuracy: 0.5819
Epoch 4/50
937/937 [==============================] - 30s 32ms/step - loss: 0.7623 - accuracy: 0.5544 - val_loss: 0.7185 - val_accuracy: 0.5881
Epoch 5/50
937/937 [==============================] - 33s 36ms/step - loss: 0.7332 - accuracy: 0.5709 - val_loss: 0.7075 - val_accuracy: 0.5634
Epoch 6/50
937/937 [==============================] - 34s 37ms/step - loss: 0.7286 - accuracy: 0.5725 - val_loss: 0.6947 - val_accuracy: 0.5868
Epoch 7/50
937/937 [==============================] - 32s 34ms/step - loss: 0.7417 - accuracy: 0.5709 - val_loss: 0.6885 - val_accuracy:

## Classification xDawn participants

In [ ]:
train_model(data_type='xDawn_participants', epochs=50, batch_size=16, multiplier=10**6)

## Classification standardization common participants

In [ ]:
train_model(data_type='standardization_common_participants', epochs=50, batch_size=16, multiplier=10)